In [1]:
import os
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
from cvzone.HandTrackingModule import HandDetector
import sys
import math
from deepmultilingualpunctuation import PunctuationModel  
sys.path.append('words')
from functions import *

In [2]:
actions = np.array(["idle","my", "name is","hello", "I", "live in"])
# letters = ['A', 'B', 'C', 'D', 'E', 'G', 'H', 'I', 'L', 'N', 'R', 'S', 'U', 'W', 'Y', 'Blank']
letters = ['A', 'D', 'N']
# letters = ['A', 'B', 'G', 'H', 'Y']
# letters = ['A', 'E', 'R', 'S', 'W']
#letters = ["A", "H", "I", "N", "R"]
# letters = ["E","L","R","U"]

In [3]:
model_LSTM = load_model('Models/model-words-2')

In [4]:
model_CNN = load_model('Models/model-4')

In [6]:
model_GRAMMER = PunctuationModel()

C:\Users\eswar\eswar7781\coding\project_test\env\Lib\site-packages\transformers\pipelines\token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.NONE"` instead.
  warnings.warn(


In [5]:
colors = [(255,255,255)] * len(actions)
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_PLAIN, 1, (255,0,0), 1, cv2.LINE_AA)
        
    return output_frame

In [6]:
def extract_features2(image):
    feature = np.array(image)
    feature = feature.reshape(1,224,224,3)
    return feature/255.0

In [ ]:
# 1. New detection variables
sequence = []
actualSentence=[]
idle_list = ["idle" for _ in range(3)]
sentence = []
predictions = []
count = 0
threshold = 0.45
i = 0
flag = 0
offset = 20
imgSize = 224
counter = 0

cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.75, min_tracking_confidence=0.75) as holistic:
    while cap.isOpened():
        if(len(sequence) == 0):
            count += 1
        
        # Read feed
        ret, img = cap.read()

        # Make detections
        if(flag == 0):
            image, results = mediapipe_detection(img, holistic)
            print(results)
            
            # Draw landmarks
            draw_styled_landmarks(image, results)
            
            # 2. Prediction logic
            if count == 5 or (len(sequence) > 0) :
                keypoints = extract_specific_keypoints(results)
                if(len(sequence) in [1,6,11]):
                    imp = important_features(image)
                    keypoints = np.concatenate([keypoints, imp])
                else:
                    keypoints = np.concatenate([keypoints, np.array([0, 0, 0, 0])])
                sequence.append(keypoints)
                count = 0        
            
            if len(sequence) == 11:
                res = model_LSTM.predict(np.expand_dims(sequence, axis=0))[0]
                print(res)
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                sequence = []
                
                if res[np.argmax(res)] > threshold:
                    actualSentence.append(actions[np.argmax(res)])  
                    
                    if len(sentence) > 0:  # Check if sentence is not empty
                        if len(sentence) >= 3:  # Check if sentence length is 3 or more
                            if actions[np.argmax(res)] != sentence[-1] and actions[np.argmax(res)] != sentence[-2] and actions[np.argmax(res)] != sentence[-3] and actions[np.argmax(res)] != "idle":
                                sentence.append(actions[np.argmax(res)])  # Append to sentence if conditions met
                        elif len(sentence) >= 2:  # Check if sentence length is 2 or more
                            if actions[np.argmax(res)] != sentence[-1] and actions[np.argmax(res)] != sentence[-2] and actions[np.argmax(res)] != "idle":
                                sentence.append(actions[np.argmax(res)])  # Append to sentence if conditions met
                        else:  # If sentence length is less than 2
                            if actions[np.argmax(res)] != sentence[-1] and actions[np.argmax(res)] != "idle":  
                                sentence.append(actions[np.argmax(res)])  # Append to sentence if conditions met
                    else:  # If sentence is empty
                        if actions[np.argmax(res)] != "idle":
                            sentence.append(actions[np.argmax(res)])  # Append to sentence if conditions met


        if(flag==1):
            imgOutput = img.copy()
            hands, image = detector.findHands(img)
            if hands:
                hand = hands[0]
                x, y, w, h = hand['bbox']
                imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
                imgCrop = image[y - offset:y + h + offset, x - offset:x + w + offset]
                imgCropShape = imgCrop.shape
                aspectRatio = h / w
                if aspectRatio > 1:
                    try:
                        k = imgSize / h
                        wCal = math.ceil(k * w)
                        imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                        imgResizeShape = imgResize.shape
                        wGap = math.ceil((imgSize - wCal) / 2)
                        imgWhite[:, wGap:wCal + wGap] = imgResize
                        imgWhite = extract_features2(imgWhite)
                        # cv2.imshow("image",imgWhite)
                        if(counter%20==0 and counter != 0):
                            counter = 1
                            pred = model_CNN.predict(imgWhite)
                            prediction_label = letters[pred.argmax()]
                            print(pred[0][np.argmax(pred)])
                            print(letters[pred.argmax()])
                            if(prediction_label != "blank"): 
                                if(len(sentence)<1 or sentence[-1] != prediction_label):
                                    print(pred)
                                    print(pred.argmax())
                                    print(pred[0][np.argmax(pred)])
                                    print(letters[pred.argmax()])
                                    if((pred[0][np.argmax(pred)]) > 0.7):
                                        sentence.append(prediction_label)
                        counter+=1
                    except Exception as e:
                        print(e)
                        continue
                else:
                    try:
                        k = imgSize / w
                        hCal = math.ceil(k * h)
                        imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                        imgResizeShape = imgResize.shape
                        hGap = math.ceil((imgSize - hCal) / 2)
                        imgWhite[hGap:hCal + hGap, :] = imgResize
                        imgWhite = extract_features2(imgWhite)
                        if(counter%20==0 and counter != 0):
                            counter = 1
                            pred = model_CNN.predict(imgWhite)
                            prediction_label = letters[pred.argmax()]
                            print(pred[0][np.argmax(pred)])
                            print(letters[pred.argmax()])
                            if(prediction_label != "blank"): 
                                if(len(sentence)<1 or sentence[-1] != prediction_label):
                                    print(pred)
                                    print(pred.argmax())
                                    print(pred[0][np.argmax(pred)])
                                    print(letters[pred.argmax()])
                                    if((pred[0][np.argmax(pred)]) > 0.7):
                                        sentence.append(prediction_label)
                                        counter = 0
                        counter+=1
                    except Exception as e:
                        print(e)
                        continue

            
        if(len(actualSentence)>=3):
            actualSentence = actualSentence[-3:] 
        
        if(actualSentence == idle_list):
            flag = 1

        cv2.putText(image," ".join(sentence), (3,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        
        cv2.putText(image, "Mode: words" if flag==0 else "Mode: letters", (image.shape[1]-640,image.shape[0]-10), 
            cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 1, cv2.LINE_AA)
        
        cv2.putText(image, str(counter) if flag==1 else str(len(sequence)), (image.shape[1]-50,image.shape[0]-10), 
            cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 1, cv2.LINE_AA)


        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('p'):
            print(model_GRAMMER.restore_punctuation(f'{" ".join(sentence)}'))

        
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q') :
            break
        
    cap.release()
    cv2.destroyAllWindows()

In [32]:
cap.release()
cv2.destroyAllWindows()
